In [1]:
import sys

# Add path for python to look into for modules installed using pip

sys.path.append('/usr/local/lib/python3.7/site-packages/')

import quandl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

quandl.ApiConfig.api_key = "n2tNssPxEFC9-Ad79fo-" # keep this private

figsize = (15, 8)

pd.set_option('display.max_columns', None)

In [2]:
# Start/end dates

start = '2010-10-15'
end = '2018-10-15'

oos_start = end
oos_end = '2019-10-15'

stock = 'BLK'

# Refinitive

### Equities Use-Cases:

* Fundamental strategies: integrate ESG factors into absolute and relative valuation models;
* Quant / Smart beta strategies: integrate ESG factors into quantitative models and adjust weights on securities based on their ESG score;
* Identify correlations between ESG scores & price movements to generate alpha and/or reduce risk.

### Alternative Use-Cases:
* Fixed-Income: Understanding how ESG issues can affect credit risk, interest rate risk, yield curve risk, and liquidity risk & affect a bond.
* Corporate Credit: Combine ESG scores with credit analysis. Assess the impact of ESG issues on a company’s ability to pay its debt obligations and liabilities.
* Structured Credit: Use ESG scores to capture risks at the transaction level (relating to the originator/ servicer/issuer of the securities), at the “collateral” or “cover” pool of underlying assets, and sometimes, informing a view on the overall deal structure.

In [3]:
# The following values are populated for you by Data Science Accelerator. 
# They represent your demo-level access to the data.
# Please don't share this with anyone

RESOURCE_ENDPOINT = 'https://dsa-stg-edp-api.fr-nonprod.aws.thomsonreuters.com/data/environmental-social-governance/v1/views/scores-full'
access_token = 'uGR7cxvvqJ4mgWwva5pPN184iGGigBhY8g4ThFu0' # your personal key for Data Science Accelerator access to ESG

def get_data_request(url, requestData):
    '''HTTP GET request'''
    dResp = requests.get(url, headers = {'X-api-key': access_token}, params = requestData);       

    if dResp.status_code != 200:
        raise ValueError("Unable to get data. Code %s, Message: %s" % (dResp.status_code, dResp.text));
    else:
        print("Data access successful")
        jResp = json.loads(dResp.text);
        return jResp

def get_data(ric):
    '''Gets ESG scores for a specific RIC (company) code'''
    
    requestData = {
    "universe": ric
    };

    jResp = get_data_request(RESOURCE_ENDPOINT, requestData)

    data = jResp["data"]
    headers = jResp["headers"]    

    names = [headers[x]['title'] for x in range(len(headers))]

    df = pd.DataFrame(data, columns=names )
    
    return df

In [4]:
# Updating the API endpoint 

RESOURCE_ENDPOINT = 'https://dsa-stg-edp-api.fr-nonprod.aws.thomsonreuters.com/data/environmental-social-governance/v1/views/measures-full'

# Get ESG scores

esg_data = get_data(stock)

# Sort ESG scores by Period End Date

esg_data = esg_data.sort_values(by='Period End Date')
esg_data['Period End Date'] = pd.to_datetime(esg_data['Period End Date'])

# Get the float64 columns that we will plot later

cols = esg_data.columns.to_series().groupby(esg_data.dtypes).groups
floatcols = cols[np.dtype('float64')]
# floatcols = floatcols[:-1] # Drop the ESG Reporting Score

esg_data.set_index(keys='Period End Date', drop=True, inplace=True)

esg_data.head() # yearly

Data access successful


,Instrument,TRESG Combined Score,TRESG Score,ESG Controversies Score,Environment Pillar Score,Social Pillar Score,Governance Pillar Score,Resource Use Score,Emissions Score,Innovation Score,Workforce Score,Human Rights Score,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score,ESG Period Last Update Date,Resource Reduction Policy,Policy Water Efficiency,Policy Energy Efficiency,Policy Sustainable Packaging,Policy Environmental Supply Chain,Resource Reduction Targets,Targets Water Efficiency,Targets Energy Efficiency,Environment Management Team,Environment Management Training,Environmental Materials Sourcing,Toxic Chemicals Reduction,Renewable Energy Use,Green Buildings,Environmental Supply Chain Management,Env Supply Chain Partnership Termination,Land Environmental Impact Reduction,Environmental Controversies,Policy Emissions,Targets Emissions,Biodiversity Impact Reduction,Estimated CO2 Equivalents Emission Total,CO2 Estimation Method,Emissions Trading,Climate Change Commercial Risks Opportunities,NOx and SOx Emissions Reduction,VOC Emissions Reduction,Particulate Matter Emissions Reduction,Waste Reduction Initiatives,e-Waste Reduction,ISO 14000 or EMS,Environmental Restoration Initiatives,Staff Transportation Impact Reduction,Environmental Expenditures Investments,Environmental Investments Initiatives,Environmental Partnerships,Environmental Products,Eco-Design Products,Noise Reduction,Hybrid Vehicles,Environmental Assets Under Mgt,Equator Principles,Equator Principles or Env Project Financing,Environmental Project Financing,Nuclear,Labeled Wood,Organic Products Initiatives,Product Impact Minimization,Take-back and Recycling Initiatives,Product Environmental Responsible Use,GMO Products,Agrochemical Products,Agrochemical 5 % Revenue,Animal Testing,Animal Testing Cosmetics,Animal Testing Reduction,Renewable/Clean Energy Products,Water Technologies,Sustainable Building Products,Product Recall,Health & Safety Policy,Policy Employee Health & Safety,Policy Supply Chain Health & Safety,Training and Development Policy,Policy Skills Training,Policy Career Development,Policy Diversity and Opportunity,Targets Diversity and Opportunity,Employees Health & Safety Team,Health & Safety Training,Supply Chain Health & Safety Training,Employees Health & Safety OHSAS 18001,Employee Satisfaction,Salary Gap,Net Employment Creation,Turnover of Employees,Announced Layoffs To Total Employees,Announced Layoffs,Management Departures,Strikes,Women Employees,New Women Employees,Women Managers,HRC Corporate Equality Index,Flexible Working Hours,Day Care Services,Employees With Disabilities,HIV-AIDS Program,Average Training Hours,Training Hours Total,Training Costs Total,Training Costs Per Employee,Internal Promotion,Management Training,Supplier ESG training,Wages Working Condition Controversies,Human Rights Policy,Policy Freedom of Association,Policy Child Labor,Policy Forced Labor,Policy Human Rights,Fundamental Human Rights ILO UN,Human Rights Contractor,Ethical Trading Initiative ETI,Human Rights Breaches Contractor,Policy Fair Competition,Policy Bribery and Corruption,Policy Business Ethics,Policy Community Involvement,Improvement Tools Business Ethics,Whistleblower Protection,OECD Guidelines for Multinational Enterprises,Extractive Industries Transparency Initiative,Lobbying Contribution Amount,Employee Engagement Voluntary Work,Corporate Responsibility Awards,Product Sales at Discount to Emerging Markets,Diseases of the Developing World,"Bribery, Corruption and Fraud Controversies",Crisis Management Systems,Anti-competition Controversies,Business Ethics Controversies,Anti-Competition Controversies Count,Critical Country 1,Critical Country 2,Recent Business Ethics Controversies,Recent Tax Fraud Controversies,Policy Customer Health & Safety,Policy Data Privacy,Policy Responsible Marketing,Policy Fair Trade,Product Responsibility Monitoring,Quality Mgt Systems,ISO 9000,Six Sigma and Quality Mgt Systems,Pr

# Quandl

In [5]:
# Stock price

stock_data = quandl.get('EOD/' + stock, start_date=start, end_date=end)

stock_data.head() # daily

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2010-10-15,176.50,177.849,175.00,175.74,339600.0,0.0,1.0,136.992729,138.039772,135.828485,136.402845,339600.0
2010-10-18,175.12,177.330,175.12,176.50,447500.0,0.0,1.0,135.921624,137.636944,135.921624,136.992729,447500.0
2010-10-19,174.75,176.000,173.00,174.63,380800.0,0.0,1.0,135.634444,136.604647,134.276159,135.541304,380800.0
2010-10-20,174.48,174.930,168.07,169.51,1435400.0,0.0,1.0,135.424880,135.774153,130.449677,131.567351,1435400.0
2010-10-21,169.98,170.070,165.17,166.26,923300.0,0.0,1.0,131.932148,132.002002,128.198805,129.044822,923300.0


In [6]:
# United States Government 2-Year Bond Yield

us_2y = quandl.get('YC/USA2Y', start_date=start, end_date=end)

# United States Government 10-Year Bond Yield

us_10y = quandl.get('YC/USA10Y', start_date=start, end_date=end)

# 10-2 Year Bond Yield Difference

two_ten_diff = us_10y['Rate'] - us_2y['Rate']

two_ten_diff = two_ten_diff.to_frame()

two_ten_diff.rename(columns={"Rate": "10Y-2Y"}, inplace=True)

two_ten_diff.head() # daily

,10Y-2Y
Date,
2010-10-15,2.22
2010-10-18,2.14
2010-10-19,2.13
2010-10-20,2.16
2010-10-21,2.20


In [7]:
# GDP

us_gdp = quandl.get('SGE/USAG', start_date=start, end_date=end)

us_gdp.rename(columns={"Value": "GDP"}, inplace=True)

us_gdp.head() # yearly

,GDP
Date,
2010-12-31,14964.4
2011-12-31,15517.9
2012-12-31,16155.3
2013-12-31,16691.5
2014-12-31,17393.1


In [8]:
# Balance of trade

us_bot = quandl.get('SGE/USABOT', start_date=start, end_date=end)

us_bot.rename(columns={"Value": "balance_of_trade"}, inplace=True)

us_bot.head() # monthly

,balance_of_trade
Date,
2010-10-31,-39998.0
2010-11-30,-37762.0
2010-12-31,-40815.0
2011-01-31,-47123.0
2011-02-28,-43718.0


In [9]:
# Government spending

us_gsp = quandl.get('SGE/USAGSP', start_date=start, end_date=end)

us_gsp.rename(columns={"Value": "government_spending"}, inplace=True)

us_gsp.head() # 3 monthly

,government_spending
Date,
2010-12-31,3071.5
2011-03-31,3012.2
2011-06-30,3009.0
2011-09-30,2990.0
2011-12-31,2978.3


In [10]:
data = stock_data.join(two_ten_diff, )

In [11]:
data = data.join(esg_data, how='left')

# data = data.join(us_bot, how='left')

# data = data.join(us_gsp, how='left')

# data = data.join(us_gdp, how='left')

In [12]:
data.head()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,10Y-2Y,Instrument,TRESG Combined Score,TRESG Score,ESG Controversies Score,Environment Pillar Score,Social Pillar Score,Governance Pillar Score,Resource Use Score,Emissions Score,Innovation Score,Workforce Score,Human Rights Score,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score,ESG Period Last Update Date,Resource Reduction Policy,Policy Water Efficiency,Policy Energy Efficiency,Policy Sustainable Packaging,Policy Environmental Supply Chain,Resource Reduction Targets,Targets Water Efficiency,Targets Energy Efficiency,Environment Management Team,Environment Management Training,Environmental Materials Sourcing,Toxic Chemicals Reduction,Renewable Energy Use,Green Buildings,Environmental Supply Chain Management,Env Supply Chain Partnership Termination,Land Environmental Impact Reduction,Environmental Controversies,Policy Emissions,Targets Emissions,Biodiversity Impact Reduction,Estimated CO2 Equivalents Emission Total,CO2 Estimation Method,Emissions Trading,Climate Change Commercial Risks Opportunities,NOx and SOx Emissions Reduction,VOC Emissions Reduction,Particulate Matter Emissions Reduction,Waste Reduction Initiatives,e-Waste Reduction,ISO 14000 or EMS,Environmental Restoration Initiatives,Staff Transportation Impact Reduction,Environmental Expenditures Investments,Environmental Investments Initiatives,Environmental Partnerships,Environmental Products,Eco-Design Products,Noise Reduction,Hybrid Vehicles,Environmental Assets Under Mgt,Equator Principles,Equator Principles or Env Project Financing,Environmental Project Financing,Nuclear,Labeled Wood,Organic Products Initiatives,Product Impact Minimization,Take-back and Recycling Initiatives,Product Environmental Responsible Use,GMO Products,Agrochemical Products,Agrochemical 5 % Revenue,Animal Testing,Animal Testing Cosmetics,Animal Testing Reduction,Renewable/Clean Energy Products,Water Technologies,Sustainable Building Products,Product Recall,Health & Safety Policy,Policy Employee Health & Safety,Policy Supply Chain Health & Safety,Training and Development Policy,Policy Skills Training,Policy Career Development,Policy Diversity and Opportunity,Targets Diversity and Opportunity,Employees Health & Safety Team,Health & Safety Training,Supply Chain Health & Safety Training,Employees Health & Safety OHSAS 18001,Employee Satisfaction,Salary Gap,Net Employment Creation,Turnover of Employees,Announced Layoffs To Total Employees,Announced Layoffs,Management Departures,Strikes,Women Employees,New Women Employees,Women Managers,HRC Corporate Equality Index,Flexible Working Hours,Day Care Services,Employees With Disabilities,HIV-AIDS Program,Average Training Hours,Training Hours Total,Training Costs Total,Training Costs Per Employee,Internal Promotion,Management Training,Supplier ESG training,Wages Working Condition Controversies,Human Rights Policy,Policy Freedom of Association,Policy Child Labor,Policy Forced Labor,Policy Human Rights,Fundamental Human Rights ILO UN,Human Rights Contractor,Ethical Trading Initiative ETI,Human Rights Breaches Contractor,Policy Fair Competition,Policy Bribery and Corruption,Policy Business Ethics,Policy Community Involvement,Improvement Tools Business Ethics,Whistleblower Protection,OECD Guidelines for Multinational Enterprises,Extractive Industries Transparency Initiative,Lobbying Contribution Amount,Employee Engagement Voluntary Work,Corporate Responsibility Awards,Product Sales at Discount to Emerging Markets,Diseases of the Developing World,"Bribery, Corruption and Fraud Controversies",Crisis Management Systems,Anti-competition Controversies,Business Ethics Controversies,Anti-Competition Controversies Count,Critical Country 1,Critical Country 2,Recent Business Ethics Controversies,Recent Tax Fraud Controversies,Policy Customer Health & Safety,Policy Data Privacy,Policy Responsible Marketing,Policy Fair Trade,Pro

In [13]:
data[data['TRESG Combined Score'].notnull()]

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,10Y-2Y,Instrument,TRESG Combined Score,TRESG Score,ESG Controversies Score,Environment Pillar Score,Social Pillar Score,Governance Pillar Score,Resource Use Score,Emissions Score,Innovation Score,Workforce Score,Human Rights Score,Community Score,Product Responsibility Score,Management Score,Shareholders Score,CSR Strategy Score,ESG Period Last Update Date,Resource Reduction Policy,Policy Water Efficiency,Policy Energy Efficiency,Policy Sustainable Packaging,Policy Environmental Supply Chain,Resource Reduction Targets,Targets Water Efficiency,Targets Energy Efficiency,Environment Management Team,Environment Management Training,Environmental Materials Sourcing,Toxic Chemicals Reduction,Renewable Energy Use,Green Buildings,Environmental Supply Chain Management,Env Supply Chain Partnership Termination,Land Environmental Impact Reduction,Environmental Controversies,Policy Emissions,Targets Emissions,Biodiversity Impact Reduction,Estimated CO2 Equivalents Emission Total,CO2 Estimation Method,Emissions Trading,Climate Change Commercial Risks Opportunities,NOx and SOx Emissions Reduction,VOC Emissions Reduction,Particulate Matter Emissions Reduction,Waste Reduction Initiatives,e-Waste Reduction,ISO 14000 or EMS,Environmental Restoration Initiatives,Staff Transportation Impact Reduction,Environmental Expenditures Investments,Environmental Investments Initiatives,Environmental Partnerships,Environmental Products,Eco-Design Products,Noise Reduction,Hybrid Vehicles,Environmental Assets Under Mgt,Equator Principles,Equator Principles or Env Project Financing,Environmental Project Financing,Nuclear,Labeled Wood,Organic Products Initiatives,Product Impact Minimization,Take-back and Recycling Initiatives,Product Environmental Responsible Use,GMO Products,Agrochemical Products,Agrochemical 5 % Revenue,Animal Testing,Animal Testing Cosmetics,Animal Testing Reduction,Renewable/Clean Energy Products,Water Technologies,Sustainable Building Products,Product Recall,Health & Safety Policy,Policy Employee Health & Safety,Policy Supply Chain Health & Safety,Training and Development Policy,Policy Skills Training,Policy Career Development,Policy Diversity and Opportunity,Targets Diversity and Opportunity,Employees Health & Safety Team,Health & Safety Training,Supply Chain Health & Safety Training,Employees Health & Safety OHSAS 18001,Employee Satisfaction,Salary Gap,Net Employment Creation,Turnover of Employees,Announced Layoffs To Total Employees,Announced Layoffs,Management Departures,Strikes,Women Employees,New Women Employees,Women Managers,HRC Corporate Equality Index,Flexible Working Hours,Day Care Services,Employees With Disabilities,HIV-AIDS Program,Average Training Hours,Training Hours Total,Training Costs Total,Training Costs Per Employee,Internal Promotion,Management Training,Supplier ESG training,Wages Working Condition Controversies,Human Rights Policy,Policy Freedom of Association,Policy Child Labor,Policy Forced Labor,Policy Human Rights,Fundamental Human Rights ILO UN,Human Rights Contractor,Ethical Trading Initiative ETI,Human Rights Breaches Contractor,Policy Fair Competition,Policy Bribery and Corruption,Policy Business Ethics,Policy Community Involvement,Improvement Tools Business Ethics,Whistleblower Protection,OECD Guidelines for Multinational Enterprises,Extractive Industries Transparency Initiative,Lobbying Contribution Amount,Employee Engagement Voluntary Work,Corporate Responsibility Awards,Product Sales at Discount to Emerging Markets,Diseases of the Developing World,"Bribery, Corruption and Fraud Controversies",Crisis Management Systems,Anti-competition Controversies,Business Ethics Controversies,Anti-Competition Controversies Count,Critical Country 1,Critical Country 2,Recent Business Ethics Controversies,Recent Tax Fraud Controversies,Policy Customer Health & Safety,Policy Data Privacy,Policy Responsible Marketing,Policy Fair Trade,Pro